In [4]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets # different version of PIL required~!
from torchvision.transforms import ToTensor

In [21]:
scalar = torch.tensor(7)
print(scalar.ndim) # 0

In [5]:
# bracket notation helps to define the dimensions of a tensor in PyTorch
x = [[1, 2],[3, 4]]

In [15]:
x_torch = torch.tensor(x)
x_np_alt = x_torch.numpy()

print("tensor: ", x_torch)
print("numpy array from tensor: ", x_np_alt)

tensor:  tensor([[1, 2],
        [3, 4]])
numpy array from tensor:  [[1 2]
 [3 4]]


In [17]:
x_np_array = np.array(x)
x_torch_alt = torch.from_numpy(x_np_array)

print("numpy array: ", x_np_array)
print("tensor from numpy array: ", x_torch_alt)

numpy array:  [[1 2]
 [3 4]]
tensor from numpy array:  tensor([[1, 2],
        [3, 4]], dtype=torch.int32)


In [20]:
print(x_torch.ndim)
print(x_torch.shape)

2
torch.Size([2, 2])


In [33]:
# detecting nvidia gpu
print("CUDA is available: ", torch.cuda.is_available())
!nvidia-smi

CUDA is available:  True
Thu Jun 22 02:31:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.03                 Driver Version: 532.03       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080       WDDM | 00000000:0A:00.0  On |                  N/A |
|  0%   49C    P8               40W / 390W|   2060MiB / 12288MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+

## Creating an image model

In [8]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:14<00:00, 1823405.95it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 104423.67it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:05<00:00, 780006.55it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 5147145.89it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [24]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [26]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [27]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [28]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [29]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [30]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300797  [   64/60000]
loss: 2.295556  [ 6464/60000]
loss: 2.277035  [12864/60000]
loss: 2.266837  [19264/60000]
loss: 2.259742  [25664/60000]
loss: 2.220728  [32064/60000]
loss: 2.226959  [38464/60000]
loss: 2.195391  [44864/60000]
loss: 2.189285  [51264/60000]
loss: 2.155452  [57664/60000]
Test Error: 
 Accuracy: 52.1%, Avg loss: 2.155563 

Epoch 2
-------------------------------
loss: 2.164619  [   64/60000]
loss: 2.156607  [ 6464/60000]
loss: 2.101626  [12864/60000]
loss: 2.112223  [19264/60000]
loss: 2.063141  [25664/60000]
loss: 2.001709  [32064/60000]
loss: 2.021379  [38464/60000]
loss: 1.946801  [44864/60000]
loss: 1.956720  [51264/60000]
loss: 1.870414  [57664/60000]
Test Error: 
 Accuracy: 57.0%, Avg loss: 1.879248 

Epoch 3
-------------------------------
loss: 1.913454  [   64/60000]
loss: 1.880282  [ 6464/60000]
loss: 1.770596  [12864/60000]
loss: 1.807312  [19264/60000]
loss: 1.696496  [25664/60000]
loss: 1.647577  [32064/600

In [34]:
# saving models
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [35]:
# loading models
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [36]:
# model evaluation
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
